In [1]:
import preprocess as pp
import configs
import persister
import relation
import lda
import json

from stanfordcorenlp import StanfordCoreNLP
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.tree import Tree

inited stanford CoreNLP client, dont forget to close it!


In [ ]:
# 文档向量化测试
# count vectorizer

In [ ]:
a = "Many existing knowledge bases(KBs), including Freebase, Yago, and NELL, rely on a ﬁxed ontology, given as an input to the system, which deﬁnes the data to be cataloged in the KB, i.e., a hierarchy of categories and relations between them. The system then extracts facts that match the predeﬁned ontology. We propose an unsupervised model that jointly learns a latent ontological structure of an input corpus, and identiﬁes facts from the corpus that match the learned structure. Our approach combines mixed membership stochastic block models and topic models to infer a structure by jointly modeling text, a latent concept hierarchy, and latent semantic relationships among the entities mentioned in the text. As a case study, we apply the model to a corpus of Web documents from the software domain,and evaluate the accuracy of the various components of the learned ontology. "
b = ["hello world. what the fuck."]

In [ ]:
c = [" ".join(preprocess(a))]
vector = CountVectorizer(ngram_range=(1, 2), vocabulary=["knowledge base"], stop_words='english')
vector.build_analyzer()
x = vector.fit_transform(c)

In [ ]:
estimator = CountVectorizer(ngram_range=(1, 2), vocabulary=['fuck', 'hello', 'world', "hello world"], stop_words='english')
estimator.build_analyzer()
res = estimator.fit_transform(b)

In [ ]:
estimator.get_feature_names()

In [ ]:
res.toarray()
# estimator.vocabulary_

In [ ]:
# 斯坦福nlp工具测试
# stanfordcorenlp

In [23]:
nlp = StanfordCoreNLP('/mnt/d/stanford-corenlp-full-2018-02-27', port=9000)
sentence = 'Brink is taking part in the final exercise.'
print('Tokenize:', nlp.word_tokenize(sentence))
print('Part of Speech:', nlp.pos_tag(sentence))
print('Named Entities:', nlp.ner(sentence))
print('Constituency Parsing:', nlp.parse(sentence))
print('Dependency Parsing:', nlp.dependency_parse(sentence))

nlp.close() # Do not forget to close! The backend server will consume a lot memery.

Tokenize: ['Brink', 'is', 'taking', 'part', 'in', 'the', 'final', 'exercise', '.']
Part of Speech: [('Brink', 'NNP'), ('is', 'VBZ'), ('taking', 'VBG'), ('part', 'NN'), ('in', 'IN'), ('the', 'DT'), ('final', 'JJ'), ('exercise', 'NN'), ('.', '.')]
Named Entities: [('Brink', 'O'), ('is', 'O'), ('taking', 'O'), ('part', 'O'), ('in', 'O'), ('the', 'O'), ('final', 'O'), ('exercise', 'O'), ('.', 'O')]
Constituency Parsing: (ROOT
  (S
    (NP (NNP Brink))
    (VP (VBZ is)
      (VP (VBG taking)
        (NP
          (NP (NN part))
          (PP (IN in)
            (NP (DT the) (JJ final) (NN exercise))))))
    (. .)))
Dependency Parsing: [('ROOT', 0, 3), ('nsubj', 3, 1), ('aux', 3, 2), ('dobj', 3, 4), ('case', 8, 5), ('det', 8, 6), ('amod', 8, 7), ('nmod', 3, 8), ('punct', 3, 9)]


In [ ]:
# 官方corenlp api
# stanza

In [ ]:
# 新闻数据集测试
# 20newsgroups

In [ ]:
import newsgroups
size = 500
tmp = newsgroups.get_news_data(size)
newsdata = []
for cate in tmp:
    newsdata.extend(cate)

In [ ]:
texts = [' '.join(pp.preprocess_abstract(a)) for a in tmp]
persister.save_json(configs.NEWSDATA, texts)

In [ ]:
# 处理成句子
# nltk.sent_tokenize

In [ ]:
newssent = []
for news in newsdata:
    newssent.append(pp.split2sent(news))
persister.save_json(configs.NEWSSENT, newssent)

In [ ]:
newssenttoken = []
for news in newssent:
    tokenized_lemmatized_news = []
    for sent in news:
        tokenized_lemmatized_news.append(relation.lemmatize_sent_words(sent))
    newssenttoken.append(tokenized_lemmatized_news)
persister.save_json(configs.NEWSSENTTOKEN, newssenttoken)

In [ ]:
# load preprossed data

In [2]:
texts = persister.load_json(configs.NEWSDATA)
newssent = persister.load_json(configs.NEWSSENT)
newssenttoken = persister.load_json(configs.NEWSSENTTOKEN)
newsparse = []
with open(configs.NEWSPARSE+".json", encoding="utf8") as f:
    for l in f:
        newsparse.append(json.loads(l))
# newsparse = [json.loads(n) for n in newsparse]

In [3]:
# load lda res
terms, doc_topic, topic_word = persister.read_lda(configs.NEWSLDA)
lda.print_topics(topic_word, terms, doc_topic)
top_terms, top_docs = lda.get_topics(topic_word, terms, doc_topic)

# 1 --------------------
window:894.4808085018936
file:844.5471131576066
image:804.220378687271
edu:771.5948685567065
graphic:586.0353569857319
program:548.6607874729475
use:548.6318542418962
server:489.24702551143844
available:488.56592120409255
display:447.98498605836124
widget:443.4712499791329
color:440.59927142836364
version:427.81261984287903
ha:413.4416084120781
application:411.2118730873431
using:390.23014165406516
com:377.30116401940575
motif:372.2449224762105
software:354.63735327422484
like:349.4803320270371
432:0.9998665847832521
160:0.9998441846371635
553:0.9998407473574282
395:0.9998369519682108
554:0.9998203062696561
60:0.9998115500043383
898:0.9998085433197132
847:0.9997970536840844
888:0.9997939325026625
520:0.9996388611461496
964:0.9995901641853403
218:0.9991375211000731
74:0.9983236847937171
55:0.9982480974139522
92:0.9982266134927563
967:0.9979275746598496
951:0.9974895529352918
154:0.9971017660445043
851:0.996842875514367
442:0.9965308830150026
# 2 ----------------

In [4]:
idx_news = 1195
idx_sent = 40
print(len(newssent[idx_news]))
print(newssent[idx_news][idx_sent])
print(newssenttoken[idx_news][idx_sent])
print(texts[idx_news])

46
My father and I built a "Veep" (Volkswagen powered Jeep CJ-2A) when I was in high school, so I consider myself fairly good with tools, electronics, and cars.
['My', 'father', 'and', 'I', 'built', 'a', '"', 'Veep', '"', '(', 'Volkswagen', 'powered', 'Jeep', 'CJ-2A', ')', 'when', 'I', 'wa', 'in', 'high', 'school', ',', 'so', 'I', 'consider', 'myself', 'fairly', 'good', 'with', 'tool', ',', 'electronics', ',', 'and', 'car', '.']
howdy little new newsgroup would like tap knowledge expertise available subject market cruise control background recently broke ankle road bicycling accident place five screw yuk two week returning texas home school byu provo utah imagine trying drive nearly mile broken right ankle epitome good time car doe cruise control would pedalling ha ha messed ankle question general opinion market cruise control unit realize cheap cc cruise control say pep boy going good factory professionally installed unit thing uderstand probably expect much way accuracy look sort thi

In [9]:
relation.get_sent_idx(newssenttoken[idx_news], "ha")

{5, 16, 21, 43}

In [5]:
print(pos_tag(word_tokenize(newssent[idx_news][43])))
print(pp.WNL.lemmatize("has", wordnet.VERB))

[(':', ':'), ('-', ':'), (')', ')'), ('I', 'PRP'), ('appreciate', 'VBP'), ('your', 'PRP$'), ('time', 'NN'), ('in', 'IN'), ('reading', 'VBG'), ('my', 'PRP$'), ('post', 'NN'), (',', ','), ('and', 'CC'), ('I', 'PRP'), ('would', 'MD'), ('appreciate', 'VB'), ('any', 'DT'), ('expertise', 'NN'), ('or', 'CC'), ('opinion', 'NN'), ('anybody', 'NN'), ('has', 'VBZ'), ('on', 'IN'), ('the', 'DT'), ('subject', 'NN'), ('.', '.')]
have


In [27]:
print(len(newsparse[idx_news]["sentences"]))
print(newsparse[idx_news]["sentences"][idx_sent].keys())
examparse = newsparse[idx_news]["sentences"][idx_sent]
print(examparse["enhancedPlusPlusDependencies"])

46
dict_keys(['index', 'parse', 'basicDependencies', 'enhancedDependencies', 'enhancedPlusPlusDependencies', 'tokens'])
[{'dep': 'ROOT', 'governor': 0, 'governorGloss': 'ROOT', 'dependent': 5, 'dependentGloss': 'built'}, {'dep': 'nmod:poss', 'governor': 2, 'governorGloss': 'father', 'dependent': 1, 'dependentGloss': 'My'}, {'dep': 'nsubj', 'governor': 5, 'governorGloss': 'built', 'dependent': 2, 'dependentGloss': 'father'}, {'dep': 'cc', 'governor': 2, 'governorGloss': 'father', 'dependent': 3, 'dependentGloss': 'and'}, {'dep': 'conj:and', 'governor': 2, 'governorGloss': 'father', 'dependent': 4, 'dependentGloss': 'I'}, {'dep': 'nsubj', 'governor': 5, 'governorGloss': 'built', 'dependent': 4, 'dependentGloss': 'I'}, {'dep': 'det', 'governor': 8, 'governorGloss': 'Veep', 'dependent': 6, 'dependentGloss': 'a'}, {'dep': 'punct', 'governor': 8, 'governorGloss': 'Veep', 'dependent': 7, 'dependentGloss': '``'}, {'dep': 'dobj', 'governor': 5, 'governorGloss': 'built', 'dependent': 8, 'depende

In [10]:
tt = Tree.fromstring(examparse["parse"])
tt.pretty_print()

                                                                                                            ROOT                                                                                                            
                                                                                                             |                                                                                                               
                                                                                                             S                                                                                                              
                                                          ___________________________________________________|____________________________________________________________________________________________________________   
                                                         S                                                        

In [6]:
t4 = [t[0] for t in top_terms[3]]
t4_values = [t[1] for t in top_terms[3]]
print(t4)
print(t4_values)
idxs, importance, count = relation.extract_important_sents(newssenttoken[idx_news], t4, t4_values)
print(idxs, importance[40], count[40])

['car', 'writes', 'article', 'wa', 'com', 'edu', 'like', 'ha', 'apr', 'know', 'engine', 'time', 'think', 'good', 'right', 'problem', 'speed', 'auto', 'really', 'doe']
[950.3249308061329, 502.30016077289747, 473.1119137059413, 432.11118671078526, 423.3220319316498, 406.1398051477658, 281.60584087001746, 223.02843201684482, 209.19909353506966, 197.8740401413088, 191.3872899764667, 184.8932378392284, 166.6527346319245, 158.1894680193873, 149.9994200649368, 143.0797041351912, 141.81018227823003, 135.03794270384222, 133.61918814004883, 133.0601717871429]
[40  5 24 22 16 21 44  0 43  4  9 10 23 26 30  7 14 35 27 20  6 15  3  8
  2  1 11 12 13 37 18 42 17 36 19 41 39 38 25 28 29 31 32 33 34 45] 1.3966649822818973 3


In [6]:
print(newssent[idx_news][idx_sent])
print(newssent[idx_news][5])

relation.extract_word_relation_from_sent(2, newsparse[idx_news]["sentences"][5]["enhancedDependencies"])

My father and I built a "Veep" (Volkswagen powered Jeep CJ-2A) when I was in high school, so I consider myself fairly good with tools, electronics, and cars.
My car does not have a cruise control, so I would have to do all the pedalling (ha ha) with my messed-up ankle.


[{'dep': 'nmod:poss',
  'governor': 2,
  'governorGloss': 'car',
  'dependent': 1,
  'dependentGloss': 'My'},
 {'dep': 'nsubj',
  'governor': 5,
  'governorGloss': 'have',
  'dependent': 2,
  'dependentGloss': 'car'}]

In [30]:
newsparse[idx_news]["sentences"][5]["enhancedDependencies"]

[{'dep': 'ROOT',
  'governor': 0,
  'governorGloss': 'ROOT',
  'dependent': 5,
  'dependentGloss': 'have'},
 {'dep': 'nmod:poss',
  'governor': 2,
  'governorGloss': 'car',
  'dependent': 1,
  'dependentGloss': 'My'},
 {'dep': 'nsubj',
  'governor': 5,
  'governorGloss': 'have',
  'dependent': 2,
  'dependentGloss': 'car'},
 {'dep': 'aux',
  'governor': 5,
  'governorGloss': 'have',
  'dependent': 3,
  'dependentGloss': 'does'},
 {'dep': 'neg',
  'governor': 5,
  'governorGloss': 'have',
  'dependent': 4,
  'dependentGloss': 'not'},
 {'dep': 'det',
  'governor': 8,
  'governorGloss': 'control',
  'dependent': 6,
  'dependentGloss': 'a'},
 {'dep': 'compound',
  'governor': 8,
  'governorGloss': 'control',
  'dependent': 7,
  'dependentGloss': 'cruise'},
 {'dep': 'dobj',
  'governor': 5,
  'governorGloss': 'have',
  'dependent': 8,
  'dependentGloss': 'control'},
 {'dep': 'punct',
  'governor': 5,
  'governorGloss': 'have',
  'dependent': 9,
  'dependentGloss': ','},
 {'dep': 'dep',
  'g

In [10]:
t = Tree.fromstring(newsparse[idx_news]["sentences"][5]["parse"])
t.pretty_print()

                                                                         ROOT                                                                                
                                                                          |                                                                                   
                                                                          S                                                                                  
                    ______________________________________________________|________________________________________________________________________________   
                   |                               |   |        S                                                                                          | 
                   |                               |   |    ____|____                                                                                      |  
                   |                             

In [ ]:
# test example res

In [ ]:
import json
with open("exampleres.json",encoding="utf8")as f:
    res = json.load(f)
print(res)